# Immersed Cylinder benchmarking



This notebook will contain all the required scripts and commands to produce the
benchmarking for the immersed cylinder case.

We expect the simulation for the `cylinder_benchmark` example have been
completed and that the resulting data is available in the `cylinder_benchmark`
subdirectory of the results folder.

![image](media/close_up.png)

The benchmarking consists of a few different sub-benchmarks:

1. Inflection point detection.
2. Backdraft region detection.
3. Leakage detection.

In [ ]:
# Assign flags for debugging to alter the behaviour of the program in addition
# to the list of experiments to compare.

debug = True
root_folder = "../../"
data_folder = "results/cylinder_benchmark"

# Define the list of data files and the labels to give them
experiments = [
    "meshed",
    "explicit_100",
    "explicit_200",
    "explicit_400",
    "explicit_800",
]
labels = [
    "Baseline",
    "Explicit 100",
    "Explicit 200",
    "Explicit 400",
    "Explicit 800",
]


In [ ]:
# Setup the paths for the project and load the external modules

# Load the external modules
import sys
import os

try:
    import import_ipynb
except:
    print("import_ipynb not installed. Installing it now.")
    %pip install import_ipynb
    import import_ipynb

# Load internal modules
try:
    from nekotop import *
except:
    sys.path.append(root_folder + "scripts/modules")
    from nekotop import *

# If the data folder is not an absolute path, make it relative to the root
if data_folder[0] != "/":
    data_folder = os.path.realpath(os.path.join(root_folder, data_folder))

# Check that the folder exists
if not os.path.exists(data_folder):
    print("The path to the results folder does not exist.")
    print("Please run the benchmark first.")
    print("\tcd NEKO_TOP_ROOT")
    print("\t./setup.sh")
    print("\t./run.sh cylinder_benchmark")
    exit(404)


# Inflection point detection

Please see the [inflection point detection notebook](inflection_points.ipynb)
for more details. Here we just execute the script and show the results.

In [ ]:
# Import the inflection benchmark and run it
#
# This cell will import and run the inflection point benchmarking for all the
# data files in the folder. The benchmark will compare the data to the baseline
# and produce a plot of the inflection points for each case.

from inflection_points import inflection_benchmark

# Define the baseline
if "baseline" not in locals() or debug:
    baseline = inflection_benchmark(
        os.path.join(data_folder, baseline_experiment, "inflection.csv"))

# Run the benchmark
if "inflection" not in locals() or debug:
    inflection = []
    for i in range(len(experiments)):
        inflection.append(
            inflection_benchmark(
                os.path.join(data_folder, experiments[i], "inflection.csv")))

In [ ]:
# Print the results in a easily readable format
def print_bias(benchmark):
    bias = benchmark["bias"]
    return f"{bias[0]:5.3f} {bias[1]:5.3f} {bias[2]:5.3f}"


def print_freq(benchmark):
    freq = benchmark["max_freq"]
    return f"{freq[0]:5.3f} {freq[1]:5.3f} {freq[2]:5.3f}"


def print_amp(benchmark):
    amp = benchmark["amplitude"]
    return f"{amp[0]:5.3f} {amp[1]:5.3f} {amp[2]:5.3f}"


def print_layer_time(benchmark):
    i = benchmark["i_boundary"]
    return f"{benchmark['times'][i]:5.3f}s"


def print_building_time(benchmark):
    i = benchmark["i_building"]
    return f"{benchmark['times'][i]:5.3f}s"


print("----------------------------------------------------------------------")
print("Bias | [rad] | [rad] | [rad]")
print("----------------------------------------------------------------------")
for i in range(len(inflection)):
    print(f"\t{labels[i]:20s}: " + print_bias(inflection[i]))
print("----------------------------------------------------------------------")
print("Frequency: [Hz]")
print("----------------------------------------------------------------------")
for i in range(len(inflection)):
    print(f"\t{labels[i]:20s}: " + print_freq(inflection[i]))
print("----------------------------------------------------------------------")
print("Amplitude: [rad]")
print("----------------------------------------------------------------------")
for i in range(len(inflection)):
    print(f"\t{labels[i]:20s}: " + print_amp(inflection[i]))
print("----------------------------------------------------------------------")
print("Layer formation time: [s]")
print("----------------------------------------------------------------------")
for i in range(len(inflection)):
    print(f"\t{labels[i]:20s}: " + print_layer_time(inflection[i]))
print("----------------------------------------------------------------------")
print("Building time: [s]")
print("----------------------------------------------------------------------")
for i in range(len(inflection)):
    print(f"\t{labels[i]:20s}: " + print_building_time(inflection[i]))
print("----------------------------------------------------------------------")

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

max_columns = 4

rows = len(experiments) // max_columns + 1
columns = min(len(experiments), max_columns)

fig, ax = plt.subplots(
    rows,
    columns,
    figsize=(6 * columns, 5 * rows),
)

(x0, x1, y0, y1) = (float("inf"), float("-inf"), float("inf"), float("-inf"))

for i in range(len(experiments)):
    idx = (i // max_columns, i % max_columns)

    # Plot the inflection angles
    ax[idx].plot(inflection[i]["times"], inflection[i]["angles"])

    # Add vertical lines to show the building and layer formation times
    ax[idx].axvline(
        inflection[i]["times"][inflection[i]["i_building"]],
        color="red",
        linestyle="--",
        label="Vortex build-up time",
    )
    ax[idx].axvline(
        inflection[i]["times"][inflection[i]["i_boundary"]],
        color="green",
        linestyle="--",
        label="Boundary formation time",
    )

    ax[idx].set_title(labels[i])

    x0 = min(x0, ax[idx].axis()[0])
    x1 = max(x1, ax[idx].axis()[1])
    y0 = min(y0, ax[idx].axis()[2])
    y1 = max(y1, ax[idx].axis()[3])

for i in range(len(experiments)):
    idx = (i // max_columns, i % max_columns)

    ax[idx].axis([x0, x1, y0, y1])

    ax[idx].set_xlabel("Time (s)")
    ax[idx].set_ylabel("Angle (rad)")
    ax[idx].legend()